In [1]:
import pandas as pd
import psycopg2
from sqlalchemy import create_engine
import numpy as np

In [2]:
def db_connection(user, password, host, port,db):
    engine= psycopg2.connect(
         user= user,
         password= password,
         host= host,
         port=port,
         database= db
    )
    return engine

In [3]:
db_engine= db_connection(user='postgres', password='postgres', host='localhost', port=5432 ,db='test')


In [4]:
def writer(user, password, host, port,db):
    engine= create_engine(
         f'postgresql://{user}:{password}@{host}:{port}/{db}'
    )
    engine.connect()
    return engine

In [5]:
db_conn= writer(user='postgres', password='postgres', host='localhost', port=5432 ,db='retaildwh')

In [6]:
# cleaned_df_new= pd.read_csv(r'C:\Users\Mohammed\Desktop\retail-DWH\data\online_retail_2.csv')
cleaned_df_new= pd.read_sql('select * from retail_cleaned', con=db_engine)
cleaned_df_new.columns= cleaned_df_new.columns.str.lower()
cleaned_df_new.head()


C:\Users\Mohammed\AppData\Local\Temp\ipykernel_17864\2770912929.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  cleaned_df_new= pd.read_sql('select * from retail_cleaned', con=db_engine)


DatabaseError: Execution failed on sql 'select * from retail_cleaned': relation "retail_cleaned" does not exist
LINE 1: select * from retail_cleaned
                      ^


In [ ]:
def dimproduct_update(new_df=cleaned_df_new, engine=db_engine):
    new_df['stockcode'] = new_df['stockcode'].astype(str)
    
    old_df = pd.read_sql('SELECT * FROM dimproduct', engine)
    old_stockcode = set(old_df['stockcode'])
    
    updated_records = new_df[new_df['stockcode'].isin(old_stockcode)]

    if not updated_records.empty:
        updated_records = updated_records[['stockcode', 'description']]\
            .drop_duplicates(subset=['stockcode', 'description'])\
            .reset_index(drop=True)
        
        with engine.cursor() as cursor:
            for index, row in updated_records.iterrows():
                update_stmt = """
                    UPDATE dimproduct
                    SET description = %s
                    WHERE stockcode = %s
                """
                cursor.execute(update_stmt, (row["description"], row["stockcode"]))
        
        engine.commit()
        print("dimproduct table updated successfully.")
        print(updated_records)
    else:
        print("No records to update.")
        
dimproduct_update()


dimproduct table updated successfully.
  stockcode                          description
0     22632            HAND WARMER RED POLKA DOT
1     22633               HAND WARMER UNION JACK
2    85123A   WHITE HANGING HEART T-LIGHT HOLDER
3     71053                  WHITE METAL LANTERN
4    84406B       CREAM CUPID HEARTS COAT HANGER
5    84029G  KNITTED UNION FLAG HOT WATER BOTTLE
6    84029E       RED WOOLLY HOTTIE WHITE HEART.
7     22752         SET 7 BABUSHKA NESTING BOXES
8     21730    GLASS STAR FROSTED T-LIGHT HOLDER


C:\Users\Mohammed\AppData\Local\Temp\ipykernel_9912\410505756.py:4: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  old_df = pd.read_sql('SELECT * FROM dimproduct', engine)


In [ ]:
def dimproduct_insert(new_df=cleaned_df_new, engine=db_engine):
    new_df['stockcode']= new_df['stockcode'].astype(str)
    
    query='''
        select * from dimproduct
    '''
    
    old_df= pd.read_sql(query, engine)
    old_stockcode= list(old_df['stockcode'])
    
    new_records= new_df[~new_df['stockcode'].isin(old_stockcode)]

    
    
    #insert new records 
    if not new_records.empty:
        new_records= new_records[['stockcode', 'description']]\
        .drop_duplicates(subset=['stockcode', 'description'])\
        .reset_index(drop=True)
        
        new_records['productid']= range(maxid+1, len(new_records)+maxid+1, 1)
        new_records['name']= 'Product-'+new_records['stockcode']
        print("data that will be inserted")
        print(new_records)
        print(new_records.dtypes)
        new_records= new_records.set_index('stockcode')
        try:
            new_records.to_sql(name='dimproduct', con=db_conn, if_exists='append', index=True, index_label='stockcode')
            print("insertion Of New Records done.")
        except Exception as e:
            print('Got error in insertion.', e)
    else:
        print("There aren't new data to be inserted.")
dimproduct_insert()

1055
There aren't new data to be inserted.


C:\Users\Mohammed\AppData\Local\Temp\ipykernel_16656\191617142.py:8: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  old_df= pd.read_sql(query, engine)
C:\Users\Mohammed\AppData\Local\Temp\ipykernel_16656\191617142.py:16: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  maxid_df= pd.read_sql(max_productid, engine)


In [ ]:
def dimcustomer_update(new_df=cleaned_df_new, engine=db_engine):

    new_df.columns= new_df.columns.str.lower()
    
    query="""
        select customerid from dimcustomer;
    """
    old_df= pd.read_sql(query, db_engine)
    old_ids= list(old_df['customerid'])
    
    updated_records= new_df[new_df['customerid'].isin(old_ids)]
    
    print(updated_records)
    if not updated_records.empty:
        with engine.cursor() as cursor:
            for index, row in updated_records.iterrows():
                update_statment='''
                    UPDATE dimcustomer
                    SET country= %s
                    WHERE customerid= %s
                '''
                cursor.execute(update_statment, (row['country'], row['customerid']))
        engine.commit()    
dimcustomer_update()

    id  invoiceno stockcode                          description  quantity  \
21  21     536372     22632            HAND WARMER RED POLKA DOT         6   
22  22     536372     22633               HAND WARMER UNION JACK         6   
23  23     536373    85123A   WHITE HANGING HEART T-LIGHT HOLDER         6   
24  24     536373     71053                  WHITE METAL LANTERN         6   
25  25     536373    84406B       CREAM CUPID HEARTS COAT HANGER         8   
26  26     536373     20679                EDWARDIAN PARASOL RED         6   
27  27     536373     37370           RETRO COFFEE MUGS ASSORTED         6   
28  28     536373     21871                  SAVE THE PLANET MUG         6   
29  29     536373     21071       VINTAGE BILLBOARD DRINK ME MUG         6   
30  30     536373     21068      VINTAGE BILLBOARD LOVE/HATE MUG         6   
31  31     536373     82483   WOOD 2 DRAWER CABINET WHITE FINISH         2   
32  32     536373     82486    WOOD S/3 CABINET ANT WHITE FINISH

C:\Users\Mohammed\AppData\Local\Temp\ipykernel_9912\1279280203.py:8: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  old_df= pd.read_sql(query, db_engine)


In [ ]:
def dimcustomer_insert(new_df=cleaned_df_new, engine=db_conn):
    new_df.columns= new_df.columns.str.lower()
    
    query="""
        select customerid from dimcustomer;
    """
    old_df= pd.read_sql(query, db_engine)
    old_ids= list(old_df['customerid'])
    
    new_records= new_df[~new_df['customerid'].isin(old_ids)]
    if not new_records.empty:
        new_records = new_records[['customerid', 'country']].copy()
        new_records['name'] = 'Customer' + new_records['customerid'].astype(str)
        new_records= new_records.reset_index(drop=True)
        new_records= new_records.drop_duplicates('customerid').set_index('customerid')
        print(new_records)
     
        new_records.to_sql(name='dimcustomer', con=engine, if_exists='append', index=True, index_label='customerid')
        print("DimCustomer data inserted successfully...")
        print(new_records)
        # except Exception as e:
        #     print("Got problem in the insertion of DimCustomer",e)
    else:
        print('there is not new data to insert')

dimcustomer_insert()

                   country           name
customerid                               
12583                Egypt  Customer12583
13748       United Kingdom  Customer13748
99999                Egypt  Customer99999
88888                Egypt  Customer88888
77777                Egypt  Customer77777
DimCustomer data inserted successfully...
                   country           name
customerid                               
12583                Egypt  Customer12583
13748       United Kingdom  Customer13748
99999                Egypt  Customer99999
88888                Egypt  Customer88888
77777                Egypt  Customer77777


C:\Users\Mohammed\AppData\Local\Temp\ipykernel_16660\1615697583.py:7: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  old_df= pd.read_sql(query, db_engine)


In [ ]:
def dimdate_insert(cleanedDF=cleaned_df_new, db_connection= ):
    query="""
        select max(invoicedate) as max_date from dimdate
    """
    
    old_max_date_df= pd.read_sql(query, engine).copy()
    old_max_date= old_max_date_df['max_date'].iloc[0]
    cleanedDF['invoicedate']= pd.to_datetime(cleanedDF['invoicedate'])
    new_max_date= cleanedDF['invoicedate'].max()
    if new_max_date> old_max_date:
        print("new is greater create a range")
        start_range= old_max_date+ pd.Timedelta(minutes=1)
        end_range= new_max_date
        print(start_range, end_range)
        Dimdate = pd.DataFrame({'InvoiceDate': pd.date_range(start=start_range, end=end_range, freq='D')})
        Dimdate['DateKey'] = Dimdate['InvoiceDate']
        Dimdate['Date'] = Dimdate['InvoiceDate'].dt.strftime('%Y-%m-%d')
        Dimdate['Year']= Dimdate['InvoiceDate'].dt.strftime('%Y')
        Dimdate['MonthNo']= Dimdate['InvoiceDate'].dt.strftime('%m')
        Dimdate['MonthName']= Dimdate['InvoiceDate'].dt.strftime('%B')
        Dimdate['Day']= Dimdate['InvoiceDate'].dt.strftime('%d')
        Dimdate['quarter']= Dimdate['InvoiceDate'].dt.quarter
        Dimdate.columns= Dimdate.columns.str.lower()
        Dimdate= Dimdate.reset_index(drop=True).set_index('datekey')
        print(Dimdate.dtypes)
        Dimdate.to_sql(name='dimdate', con=db_connection ,if_exists='append', index=True, index_label='datekey')
    else:
        print("no updates in date")
        

dimdate_insert()

C:\Users\Mohammed\AppData\Local\Temp\ipykernel_16660\2950772273.py:6: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  old_max_date_df= pd.read_sql(query, engine).copy()


new is greater create a range
2010-12-01 08:27:00 2013-12-01 08:45:00
invoicedate    datetime64[ns]
date                   object
year                   object
monthno                object
monthname              object
day                    object
quarter                 int64
dtype: object


In [ ]:
# saleskey| invoiceno | datekey | customerkey | productkey | unitprice | quantity
clened_data= """
    SELECT c.saleskey,
        c.invoiceno,
        c.invoicedate as datekey,
        c.customerid as customerkey,
        c.stockcode as productkey,
        c.unitprice,
        c.quantity 
    FROM retail_cleaned c INNER JOIN dimproduct p
    ON c.stockcode= p.stockcode
    AND c.description = p.description

    INNER JOIN dimcustomer cust 
    ON c.customerid= cust.customerid
"""
insert_fact= pd.read_sql(clened_data, con=db_engine)
insert_fact.head()

C:\Users\Mohammed\AppData\Local\Temp\ipykernel_16660\3601415959.py:17: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  insert_fact= pd.read_sql(clened_data, con=db_engine)


,saleskey,invoiceno,datekey,customerkey,productkey,unitprice,quantity
0,125832272820101201,536370,2010-12-01 08:45:00,12583,22728,3.75,24
1,125832272720101201,536370,2010-12-01 08:45:00,12583,22727,3.75,24
2,125832272620101201,536370,2010-12-01 08:45:00,12583,22726,3.75,12
3,125832172420101201,536370,2010-12-01 08:45:00,12583,21724,0.85,12
4,125832188320101201,536370,2010-12-01 08:45:00,12583,21883,0.65,24


In [ ]:
insert_fact['invoiceno'].count()

53

In [ ]:
old_fact= pd.read_sql('select * from fact_sales', con=db_engine)

C:\Users\Mohammed\AppData\Local\Temp\ipykernel_16660\992800974.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  old_fact= pd.read_sql('select * from fact_sales', con=db_engine)


In [ ]:
old_fact['customerkey'].count()

21

In [ ]:
df= insert_fact.append(old_fact)

C:\Users\Mohammed\AppData\Local\Temp\ipykernel_16660\4121976351.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df= insert_fact.append(old_fact)


In [ ]:
df.head()

,saleskey,invoiceno,datekey,customerkey,productkey,unitprice,quantity
0,125832272820101201,536370,2010-12-01 08:45:00,12583,22728,3.75,24
1,125832272720101201,536370,2010-12-01 08:45:00,12583,22727,3.75,24
2,125832272620101201,536370,2010-12-01 08:45:00,12583,22726,3.75,12
3,125832172420101201,536370,2010-12-01 08:45:00,12583,21724,0.85,12
4,125832188320101201,536370,2010-12-01 08:45:00,12583,21883,0.65,24


In [ ]:
df['customerkey'].count()

74

In [ ]:
df.duplicated().sum()

0

In [ ]:
def laod_fact(engine= db_engine, db_connection= db_conn):
    # saleskey| invoiceno | datekey | customerkey | productkey | unitprice | quantity
    clened_data= """
        SELECT c.saleskey,
            c.invoiceno,
            c.invoicedate as datekey,
            c.customerid as customerkey,
            c.stockcode as productkey,
            c.unitprice,
            c.quantity 
        FROM retail_cleaned c INNER JOIN dimproduct p
        ON c.stockcode= p.stockcode
        AND c.description = p.description

        INNER JOIN dimcustomer cust 
        ON c.customerid= cust.customerid
    """
    fact= pd.read_sql(clened_data, con=engine)
    fact= fact.reset_index(drop=True).set_index('saleskey')
    print(insert_fact.head())
    df.to_sql(name='fact_sales', con=db_connection, if_exists='append',index=True, index_label='saleskey')
    print('done')
laod_fact()

C:\Users\Mohammed\AppData\Local\Temp\ipykernel_16660\824621770.py:18: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  fact= pd.read_sql(clened_data, con=engine)


             saleskey  invoiceno             datekey  customerkey productkey  \
0  125832272820101201     536370 2010-12-01 08:45:00        12583      22728   
1  125832272720101201     536370 2010-12-01 08:45:00        12583      22727   
2  125832272620101201     536370 2010-12-01 08:45:00        12583      22726   
3  125832172420101201     536370 2010-12-01 08:45:00        12583      21724   
4  125832188320101201     536370 2010-12-01 08:45:00        12583      21883   

   unitprice  quantity  
0       3.75        24  
1       3.75        24  
2       3.75        12  
3       0.85        12  
4       0.65        24  


DuplicateColumnError: A column with name 'saleskey' is already present in table 'fact_sales'.

In [ ]:
db_engine= db_connection(user='postgres', password='postgres', host='localhost', port=5432 ,db='test')
cur = db_engine.cursor()

with open('../ETL-SCD-Queries/dimproduct.sql', 'r') as file:
    sql_query = file.read()
cur.execute(sql_query)
results = cur.fetchall()
cur.close()
db_engine.close()
df = pd.DataFrame(results, columns=['stockcode', 'description'])
print(df)


UndefinedTable: relation "retail_cleaned" does not exist
LINE 9:         FROM retail_cleaned
                     ^


In [ ]:
SELECT stockcode, description 
FROM (
    SELECT stockcode, description, processed_date, 
           ROW_NUMBER() OVER (PARTITION BY stockcode ORDER BY processed_date DESC) AS row_num
    FROM (
        SELECT stockcode,
        description,
        DATE_ADD(CURRENT_DATE, INTERVAL 1 DAY) AS processed_date
        FROM retail_cleaned
        
        UNION ALL
        SELECT stockcode,
        description,
        processed_date
        FROM dimproduct
    ) AS combined
) AS numbered_rows
WHERE row_num = 1

UNION ALL

SELECT stockcode, description 
FROM retail_cleaned 
WHERE stockcode NOT IN (SELECT stockcode FROM dimproduct);
